In [1]:
import pandas as pd
import gc

In [2]:
# sunat_train = pd.read_csv('../data/sunat_train.csv')
rcc_train = pd.read_csv('../data/rcc_train.csv')
# se_train = pd.read_csv('../data/se_train.csv', index_col = 'key_value')
# censo_train = pd.read_csv('../data/censo_train.csv', index_col = 'key_value')
y_train = pd.read_csv('../data/y_train.csv', index_col = 'key_value')

# sunat_test = pd.read_csv('../data/sunat_test.csv')
rcc_test = pd.read_csv('../data/rcc_test.csv')
# se_test = pd.read_csv('../data/se_test.csv', index_col = 'key_value')
# censo_test = pd.read_csv('../data/censo_test.csv', index_col = 'key_value')
sample_submision = pd.read_csv('../data/sample_submission.csv', index_col = 'key_value')

In [3]:
bins = [-1, 0, 10, 20, 30, 60, 90, 180, 360, 720, float("inf")]
rcc_train["condicion"] = pd.cut(rcc_train.condicion, bins, labels = range(0,10))
rcc_test["condicion"] = pd.cut(rcc_test.condicion, bins, labels = range(0,10))

In [4]:
rcc_test['cod_instit_financiera'] = rcc_test['cod_instit_financiera'].fillna(34)
rcc_test['PRODUCTO'] = rcc_test['PRODUCTO'].fillna(1)

In [5]:
rcc_train = rcc_train.astype({'condicion':'int32',
                  'tipo_credito':'int32',
                  'cod_instit_financiera':'int32',
                  'PRODUCTO':'int32',
                  'RIESGO_DIRECTO':'int32',
                  'COD_CLASIFICACION_DEUDOR':'int32'})
rcc_test = rcc_test.astype({'condicion':'int32',
                  'tipo_credito':'int32',
                  'cod_instit_financiera':'int32',
                  'PRODUCTO':'int32',
                  'RIESGO_DIRECTO':'int32',
                  'COD_CLASIFICACION_DEUDOR':'int32'})

In [6]:
from tqdm.notebook import tqdm 
list_rcc_train_agg = []
for n,i in enumerate(tqdm(sorted(set(rcc_train.codmes),reverse=True))):
    if i in [201802,201712,201709,201703]:
        for c in ['tipo_credito','cod_instit_financiera','PRODUCTO','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR','condicion']:
            rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby(['key_value', c]).saldo.sum().unstack(level = 1, fill_value=0)
            rcc_train_agg.columns = [f'{rcc_train_agg.columns.name}_{c}_ult{n+1}meses' for c in rcc_train_agg.columns]
            list_rcc_train_agg.append(rcc_train_agg)
        gc.collect()

In [7]:
rcc_train_ = pd.concat(list_rcc_train_agg, axis=1)
del rcc_train, list_rcc_train_agg

In [8]:
from tqdm.notebook import tqdm 
list_rcc_test_agg = []
for n,i in enumerate(tqdm(sorted(set(rcc_test.codmes),reverse=True))):
    if i in [201902,201812,201809,201803]:
        for c in ['tipo_credito','cod_instit_financiera','PRODUCTO','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR','condicion']:
            rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby(['key_value', c]).saldo.sum().unstack(level = 1, fill_value=0)
            rcc_test_agg.columns = [f'{rcc_test_agg.columns.name}_{c}_ult{n+1}meses' for c in rcc_test_agg.columns]
            list_rcc_test_agg.append(rcc_test_agg)
        gc.collect()

In [9]:
rcc_test_ = pd.concat(list_rcc_test_agg, axis=1)
del rcc_test, list_rcc_test_agg

In [10]:
se_train = pd.read_csv('../data/se_train.csv', index_col = 'key_value')
# censo_train = pd.read_csv('../data/censo_train.csv', index_col = 'key_value')
se_test = pd.read_csv('../data/se_test.csv', index_col = 'key_value')
# censo_test = pd.read_csv('../data/censo_test.csv', index_col = 'key_value')

In [11]:
gc.collect()

70

In [12]:
train = rcc_train_.join(se_train)#.join(censo_train).join(se_train)
test = rcc_test_.join(se_test)#join(censo_test).join(se_test)

In [13]:
train.shape, test.shape

((358487, 711), (396666, 663))

In [14]:
del se_train, se_test
gc.collect()

44

In [15]:
keep_cols = list(set(train.columns).intersection(set(test.columns)))
train = train[keep_cols].copy()
test = test[keep_cols].copy()
len(set(train.columns) - set(test.columns)) , len(set(test.columns) - set(train.columns))

(0, 0)

In [16]:
import re
test = test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_-]+', '', x))
train = train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_-]+', '', x))

In [17]:
#Función para obtener variables con alta correlación
def get_correlated(data, cut=0.85):
    correlated_features = set()
    correlation_matrix = data.corr()
    
    for i in range(len(correlation_matrix .columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) > cut:
                colname = correlation_matrix.columns[i]
                correlated_features.add(colname)
            
    return correlated_features

In [18]:
train.shape

(358487, 656)

In [19]:
correlated=get_correlated(train,cut=0.9)
train=train.drop(columns=correlated)
test= test.drop(columns=correlated)

In [20]:
train.shape

(358487, 276)

In [21]:
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

folds = [train.index[t] for t, v in KFold(5).split(train)]
test_probs = []
train_probs = []
fi = []
for i, idx in enumerate(folds):
    print("*"*10, i, "*"*10)
    Xt = train.loc[idx]
    yt = y_train.loc[Xt.index]

    Xv = train.drop(Xt.index)
    yv = y_train.loc[Xv.index]
    
    learner = LGBMClassifier(n_estimators=1000)
    learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
                eval_set=[(Xt, yt), (Xv, yv)], verbose=50)
    test_probs.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
    train_probs.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))
          
test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
train_probs = pd.concat(train_probs)
fi = pd.concat(fi, axis=1).mean(axis=1)

print("*" * 21)
print("roc auc estimado: ", roc_auc_score(y_train, train_probs.loc[y_train.index]))

********** 0 **********


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.838991	training's binary_logloss: 0.305793	valid_1's auc: 0.830297	valid_1's binary_logloss: 0.311855
[100]	training's auc: 0.853055	training's binary_logloss: 0.294344	valid_1's auc: 0.839922	valid_1's binary_logloss: 0.304122
[150]	training's auc: 0.861475	training's binary_logloss: 0.287972	valid_1's auc: 0.84202	valid_1's binary_logloss: 0.302423
Early stopping, best iteration is:
[159]	training's auc: 0.862761	training's binary_logloss: 0.287003	valid_1's auc: 0.842197	valid_1's binary_logloss: 0.302227
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.839038	training's binary_logloss: 0.305773	valid_1's auc: 0.832348	valid_1's binary_logloss: 0.310717
[100]	training's auc: 0.853014	training's binary_logloss: 0.294213	valid_1's auc: 0.841155	valid_1's binary_logloss: 0.303426
[150]	training's auc: 0.861305	training's binary_logloss: 0.287968

In [23]:
fi.sort_values(ascending=False).head(50)

PRODUCTO_1_ult1meses                     0.045238
tipo_credito_11_ult12meses               0.027882
PRODUCTO_3_ult3meses                     0.027279
edad                                     0.025727
tipo_credito_11_ult1meses                0.022727
PRODUCTO_0_ult3meses                     0.022691
PRODUCTO_6_ult1meses                     0.022666
COD_CLASIFICACION_DEUDOR_5_ult1meses     0.022271
PRODUCTO_8_ult3meses                     0.019368
PRODUCTO_1_ult6meses                     0.018764
cod_ubi                                  0.018428
PRODUCTO_4_ult1meses                     0.017753
PRODUCTO_11_ult1meses                    0.016435
RIESGO_DIRECTO_1_ult6meses               0.015871
tip_lvledu                               0.014463
tipo_credito_12_ult1meses                0.014202
PRODUCTO_7_ult12meses                    0.013851
dto                                      0.013592
cod_instit_financiera_34_ult3meses       0.012068
cod_instit_financiera_32_ult12meses      0.011984


In [ ]:
# test_probs.name = "target"
# test_probs.to_csv("../results/test4.csv")

## Probando con CatBoost

In [24]:

# !pip install catboost
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

folds = [train.index[t] for t, v in KFold(5).split(train)]
test_probs = []
train_probs = []
fi = []
for i, idx in enumerate(folds):
    print("*"*10, i, "*"*10)
    Xt = train.loc[idx]
    yt = y_train.loc[Xt.index]

    Xv = train.drop(Xt.index)
    yv = y_train.loc[Xv.index]
    
    learner = CatBoostClassifier(n_estimators=1000, depth = 6, eval_metric = 'AUC' )
    learner.fit(Xt, yt,  early_stopping_rounds=10,
                eval_set=[(Xt, yt), (Xv, yv)], verbose=50)
    test_probs.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
    train_probs.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))
          
test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
train_probs = pd.concat(train_probs)
fi = pd.concat(fi, axis=1).mean(axis=1)

print("*" * 21)
print("roc auc estimado: ", roc_auc_score(y_train, train_probs.loc[y_train.index]))

********** 0 **********
Learning rate set to 0.128212
0:	test: 0.6949396	test1: 0.6991357	best: 0.6991357 (0)	total: 257ms	remaining: 4m 16s
50:	test: 0.8192151	test1: 0.8165845	best: 0.8165845 (50)	total: 10.3s	remaining: 3m 12s
100:	test: 0.8308329	test1: 0.8272455	best: 0.8272455 (100)	total: 20.1s	remaining: 2m 59s
150:	test: 0.8387963	test1: 0.8333692	best: 0.8333692 (150)	total: 30.3s	remaining: 2m 50s
200:	test: 0.8437851	test1: 0.8360357	best: 0.8360357 (200)	total: 40.2s	remaining: 2m 39s
250:	test: 0.8475939	test1: 0.8379175	best: 0.8379175 (250)	total: 49.7s	remaining: 2m 28s
300:	test: 0.8505720	test1: 0.8391934	best: 0.8391934 (300)	total: 59s	remaining: 2m 17s
350:	test: 0.8537054	test1: 0.8402310	best: 0.8402310 (350)	total: 1m 8s	remaining: 2m 6s
Stopped by overfitting detector  (10 iterations wait)

bestTest = 0.8407104114
bestIteration = 372

Shrink model to first 373 iterations.
********** 1 **********
Learning rate set to 0.128212
0:	test: 0.6927740	test1: 0.6957486

In [27]:
fi.sort_values(ascending=False).tail(50)#.plot.bar()

cod_instit_financiera_79_ult12meses    6.897137e-07
PRODUCTO_37_ult12meses                 2.495657e-07
cod_instit_financiera_88_ult1meses     2.163555e-07
cod_instit_financiera_70_ult1meses     2.125856e-07
cod_instit_financiera_78_ult3meses     9.767097e-08
PRODUCTO_34_ult12meses                 4.779898e-08
tipo_credito_6_ult6meses               1.117143e-08
cod_instit_financiera_100_ult6meses    0.000000e+00
cod_instit_financiera_26_ult12meses    0.000000e+00
cod_instit_financiera_71_ult1meses     0.000000e+00
PRODUCTO_41_ult12meses                 0.000000e+00
cod_instit_financiera_85_ult12meses    0.000000e+00
cod_instit_financiera_14_ult12meses    0.000000e+00
cod_instit_financiera_90_ult12meses    0.000000e+00
tipo_credito_6_ult12meses              0.000000e+00
cod_instit_financiera_76_ult1meses     0.000000e+00
cod_instit_financiera_89_ult6meses     0.000000e+00
PRODUCTO_27_ult1meses                  0.000000e+00
cod_instit_financiera_105_ult1meses    0.000000e+00
PRODUCTO_32_

In [26]:
test_probs.name = "target"
test_probs.to_csv("../results/test4.csv")